### Parametros do inversor


| Parâmetro | Valor |
|----------|----------|
| Potencia aparente (S)| 1kVA | 
| Tensão (Vc = Vg)| 127V|
| Frequência (fg) | 60Hz |
| Corrente máxima ($I_{max}$)  | 11.135A $_{peak}$ |
| Ripple máximo ($\Delta I_{max}$) | 1.13A |
| Topologia da ponte | Full Bridge |
| Indice de modulação | 0.8 |
| Tensão CC ($V_{DC}$) | 225V |
| Freq. chaveamento(fsw) | 24kHz |
| Freq. amostragem(fs) | 48kHz |


### Dimensionando número de espiras

Para a construção do indutor, começamos com um nucleo já pré determinado e fornecido. Iremos usar o indutor de _part number_ [MMT034T10233](https://www.magmattec.com.br/wp-content/uploads/2019/03/MATERIAL-034_01.pdf).

Para a construção do indutor de 300uH, iremos usar a formula a seguir, que nos fornece o numero de espiras com base na indutância desejada e no parametro _Al_ disponivel no _datasheet_ do nucleo.

$$N=\sqrt{\frac{L}{Al}} \tag{1}$$

De acordo com o datasheet, $Al=110nH/esp^2$, assim teremos;

In [1]:
import math
Al = 110e-9
L = 300e-6

In [2]:
N = math.sqrt(L/Al)
N

52.22329678670935

Vamos arredondar o **número de espiras para 52** e seguir com a construção do indutor. A próxima fórmula que iremos usar também serve para encontrar o número de espiras, porém, usaremos para encontrar o valor da densidade de campo $B_m$.

$$N=\frac{L \cdot I_{pico}}{B_m \cdot A_e} \cdot 10^4 \tag{2}$$

O fator $10^4$ usado na equação serve para que possamos usar a área de secção do núcleo em $cm^2$. Caso o valor esteja em $m^2$, não é necessário o uso do fator multiplicativo.

A corrente de pico que iremos usar, será a corrente máxima somada à metade do ripple de corrente,


In [3]:
I_max = 11.135
I_ripple = 1.13

I_pico = I_max + I_ripple/2
I_pico

11.7

A corrente de pico será então equivalente a $11.7\,A$. 

A área efetiva do núcleo toroidal é dada pela folha de dados do fabricante e, para o nosso caso, essa tem o valor de 6.85 $cm^2$

Resolvendo para $B_m$ em (2), o valor de densidade de campo será,

In [8]:
import sympy as sp

A_e = 6.85
B_m = sp.symbols("B_m")

B_m = sp.solve(sp.Eq((L*I_pico*10**4)/(B_m*A_e),N), B_m)
B_m

[0.0981188072474380]

O valor de densidade de campo necessária será de $0.0981\,T$


Após consultar a folha de dados do núcleo em uso, identificamos que a densidade de campo máxima $B_{max}$​ é de 10 000 Gauss, o que corresponde a 1.1 Tesla. Portanto, estamos operando dentro de uma faixa de densidade de campo que o núcleo pode suportar, sem entrar na região de saturação. É recomendável que esta utilização fique entre 50% e 70% da capacidade máxima do núcleo, para evitar qualquer problema que operar em saturação possa causar. No nosso caso, estamos utilizando uma proporção de uso de:

In [16]:
B_m_max = 1.1

B_relation_to_max = float(B_m[0]/1.1)
B_relation_to_max


0.08919891567948911

A faixa de uso em relação a $B_{max}$ é de apenas $8.9\%$, ou seja, nosso nucleo está super dimensionado para a nossa aplicação. De qualquer forma, isso não irá causar problemas.

Prosseguindo com os cálculos, iremos agora calcular a área da janela $(A_j)$ usada pelo indutor. Vamos considerar o número de espiras $(N)$ encontrado, uma densidade de corrente máxima $(J_{máx})$ de 4.5 $A/mm^2$. O fator de ocupação $(K_w)$ para núcleos toroidais fica abaixo de 0.3. Usaremos $K_w = 0.2$. Assim, teremos para $A_j$:

$$
A_j = \frac{N \cdot I_{ef}}{J_{max} \cdot K_w} \tag{3}
$$

In [10]:
K_w = 0.2
J_max =  4.5/0.01 #converting from A/mm^2 to A/cm^2
I_ef = I_pico/math.sqrt(2)

Aj = (N*I_ef)/(J_max*K_w)
Aj

4.8005681481939755

Temos assim, $A_j = 4.8 cm^2$.
No datasheet do núcleo, podemos verificar a área de janela disponível, uma vez que temos acesso ao raio interno do toróide, 5.72 $cm$, temos assim:

In [17]:
math.pi * 5.72**2

102.78788507721227

A área de janela disponível $(A_{j\:disponivel})$ do nucleo toroidal que possuímos é de $102.78\:cm^2$. 

Podemos concluir então que, a área disponível, é mais que suficiente para a realização do enrolamento do fio no núcleo fornecido. $A_{j\:usado} < A_{j\:disponivel}$

### Dimensionamento do condutor

O condutor a ser usado na construção do indutor precisará ter secção capaz de suportar a corrente de operação do circuito. Para isso usaremos a fórmula (4) para encontrar qual a secção ideal deste fio,

$$S_f = \frac{I_{ef}}{J_{máx}} \tag{4}$$

In [12]:
S_f = I_ef/(J_max*0.01) #multiplying by 0.01 to convert from cm^2 to mm^2
S_f

1.8384776310850235

A secção ideal deste fio é então $S_f = 1.83\:mm^2$.

O próximo passo é aferir qual o impacto da frequência de chaveamento na seleção deste condutor. Para isso iremos usar a fórmula (5) para saber qual profundidade a corrente irá fluir no fio. Caso a secção deste condutor seja menor do que $S_f$, será necessário utilizar mais de um condutor em paralelo até atingir uma secção igual ou superior a de $S_f$.

$$\gamma = \sqrt{\frac{4.35 \cdot 10^{-3}}{f}} \tag{5}$$

In [13]:
freq = 48e3
gamma = math.sqrt(4.35e-3/freq)
gamma

0.0003010398644698074

Obtemos então que, $\gamma = 0.3\:mm$. Isso implica que, podemos usar um fio com diâmetro de $0.6\,mm$ sem desperdício algum de material.

Tivemos acesso à um fio esmaltado de 18 AWG. De acordo com as tabelas consultadas online, o fio possui diametro de $1.024\:mm$. Assim, haverá uma pequena região circular, no interior deste fio, que não será de fato útil para a passagem de corrente em $48kHz$, porém, como material e dimensões não são restrições para o projeto, podemos seguir sem maiores problemas.

Usando o diâmetro do cabo de 18AWG encontrado para obter a secção, temos:

In [14]:
math.pi*(1.024/2)**2

0.8235496645826427

secção do cabo de 18 AWG é de $0.823\:mm^2$.

Portanto, para este fio, conseguimos conhecer o número de condutores em paralelo $(ncp)$ necessário para a construção do indutor. Calculamos $ncp$ a seguir:

$$ncp = \frac{S_f}{S_{18\:awg}} \tag{6} $$

In [15]:
ncp = S_f/(math.pi*(1.024/2)**2)
ncp

2.2323822231373556

Com $ncp = 2.23$, podemos concluir que, para obter a secção de condutor ideal precisaremos usar 3 fios condutores e 52 voltas em torno do núcleo toroidal fornecido para a construção do indutor de 300uH.